## Data Ingestion stage

In [1]:
from collections import namedtuple
import os

In [2]:
DataIngestionConfig = namedtuple("DataIngestionConfig",[
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile

In [18]:
import os
import zipfile
from urllib.error import URLError, HTTPError
from urllib import request

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                print("Downloading file...")
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                print("Download complete.")
            else:
                print("File already exists. Skipping download.")
        except (URLError, HTTPError) as e:
            raise RuntimeError(f"Failed to download file from {self.config.source_URL}: {e}")

    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self, zf: zipfile.ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:  # Remove empty files
            os.remove(target_filepath)

    def unzip_and_clean(self):
        try:
            print("Unzipping file...")
            with zipfile.ZipFile(file=self.config.local_data_file, mode="r") as zf:
                list_of_files = zf.namelist()
                updated_list_of_files = self._get_updated_list_of_files(list_of_files)
                for f in updated_list_of_files:
                    self._preprocess(zf, f, self.config.unzip_dir)
            print("Unzipping and cleaning complete.")
        except zipfile.BadZipFile:
            raise RuntimeError(f"The file {self.config.local_data_file} is not a valid zip file.")
        except Exception as e:
            raise RuntimeError(f"An error occurred while unzipping the file: {e}")


In [ ]:
os.chdir("../")
%pwd

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e